<a href="https://colab.research.google.com/github/barbaroja2000/llm/blob/main/Hugging_Face_%7C_Transformers_Quantized_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers Quantized Models
Works for 3b ono size models in Colab with T4|v100,  any bigger than that and session crashes.

https://huggingface.co/blog/4bit-transformers-bitsandbytes
https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U torch

In [ ]:
from torch import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
#@title Load Keys
#@markdown Utitily to load keys from fs, replace with environ vars if not using

import os

#os.environ.get("HUGGINGFACEHUB_API_TOKEN")

!python -m pip install python-dotenv
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
import dotenv
dotenv.load_dotenv('/content/drive/MyDrive/keys/keys.env')

os.environ["HUGGINGFACEHUB_API_TOKEN"]= os.environ.get("HUGGINGFACE_API_KEY")

In [ ]:
#@title Sets Transformers cache
#@markdown Model reuse
os.environ['TRANSFORMERS_CACHE'] = '/content/drive/MyDrive/models/hf/'

!ls -la $TRANSFORMERS_CACHE

In [ ]:
#10GB
#https://huggingface.co/EleutherAI/gpt-neo-2.7B
name="EleutherAI/gpt-neo-2.7B"

In [ ]:
#@title Quantization

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model_nf4 = AutoModelForCausalLM.from_pretrained(name, quantization_config=nf4_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(name)

In [ ]:
#got gpu
model_nf4.hf_device_map

In [ ]:
text = """Things like gravy go right in among the hairs and stay there. You and
I can wipe our smooth faces with a flannel and we quickly look more or less
all right again, but the hairy man cannot do that.
We can also, if we are careful, eat our meals without spreading food all
over our faces. But not so the hairy man. Watch carefully next time you see
a hairy man eating his lunch and you will notice that even if he opens his
mouth very wide, it is impossible for him to get a spoonful of beef-stew or
ice-cream and chocolate sauce into it without leaving some of it on the
hairs.
Mr Twit didn't even bother to open his mouth wide when he ate. As a result
(and because he never washed) there were always hundreds of bits of old
breakfasts and lunches and suppers sticking to the hairs around his face.
They weren't big bits, mind you, because he used to wipe those off with the
back of his hand or on his sleeve while he was eating. But if you looked
closely (not that you'd ever want to) you would see tiny little specks of
dried-up scrambled eggs stuck to the hairs, and spinach and tomato
ketchup and fish fingers and minced chicken livers and all the other
disgusting things Mr Twit liked to eat.
If you looked closer still (hold your noses, ladies and gentlemen), if you
peered deep into the moustachy bristles sticking out over his upper lip, you would
probably see much larger objects that had escaped the wipe of his hand,
things that had been there for months and months, like a piece of maggoty
green cheese or a mouldy old cornflake or even the slimy tail of a tinned
sardine.
Because of all this, Mr Twit never went really hungry. By sticking out his
tongue and curling it sideways to explore the hairy jungle around his mouth,
he was always able to find a tasty morsel here and there to nibble on.
What I am trying to tell you is that Mr Twit was a foul and smelly old man.
He was also an extremely horrid old man, as you will find out in a moment."""
device = "cuda:0"
tokenizer = AutoTokenizer.from_pretrained(name)

#https://huggingface.co/docs/transformers/main/generation_strategies

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model_nf4.generate(**inputs, do_sample=True, num_beams=1, max_new_tokens=1000)
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
print("\n".join(output_text))                                                                   